In [0]:
# imports

import json
from utils.api import buscar_dados, obter_estado_id_por_uf
from pyspark.sql.functions import current_timestamp, lit
import pandas as pd
 

In [0]:
#Parametros para facilitar a execução e reutilização do notebook para carga de diferentes estados
dbutils.widgets.text("uf", "MG", "UF")

In [0]:
#Variaveis do Unity Catalog
CATALOGO = "agroclima"
SCHEMA = "bronze"
TABELA = "municipios"

# Montar caminho da tabela
caminho_tabela = f"{CATALOGO}.{SCHEMA}.{TABELA}"



In [0]:
# Transforma os parametros em variaveis
uf = dbutils.widgets.get("uf")

In [0]:
# Seta Variavel com o Id da UF 
estado_id = obter_estado_id_por_uf(uf)


In [0]:
# Função que consome a API do IBGE e retorna dados dos municipios
def obter_municipios_por_estado(estado_id):
       
    # Cria URL com base nos parametros informados
    IBGE_URL = f"https://servicodados.ibge.gov.br/api/v1/localidades/estados/{estado_id}/municipios"
    
    # Executa API do IBGE buscando dados dos municipios
    dados_municipios = buscar_dados(IBGE_URL)

    #Transforma Json em String para ser armazenado no dataframe
    df_nrmalizado = pd.json_normalize(dados_municipios)
    
    #Cria Dataframe com dados do Json
    df_municipios = spark.createDataFrame(df_nrmalizado)


    #Adiciona colunas para controle de carga
    df_municipios = df_municipios.withColumn("uf", lit(uf)) \
                                   .withColumn("_data_ingestao", current_timestamp())

    return df_municipios


In [0]:

try:
    # Chama Função que consome a API com cada paramtero de Ano da Lista passada em parametro
    df_municipios = obter_municipios_por_estado( estado_id)

    #Cria Tabela Delta no Unity Catalog  atualizando somente ano e uf passado em parametro
    df_municipios.write.format("delta") \
        .mode("overwrite") \
        .partitionBy("uf") \
        .option("replaceWhere", f"uf = '{uf}'") \
        .option("mergeSchema", "true") \
        .saveAsTable(caminho_tabela)
except Exception as e:
    pass